# Notebook 02: Calcular Métricas
Este notebook calcula métricas dos experimentos A/B/C por grupo e replica para cada cliente

In [ ]:
# Imports e configurações
import boto3
import pandas as pd
import json
from datetime import datetime

# Parâmetros do pipeline
bucket_name = "experimento-lucas-barbosa"
user = "lucas-barbosa"
prefixo_simulado = "pipeline-experimentos"
input_config = "config.json"
input_validation = "validacao_resultado.json"
output_metricas = "metricas_completas.csv"

# Cliente S3
s3 = boto3.client('s3')

In [ ]:
# Carregar configurações
config_path = f"{user}/{prefixo_simulado}/config/{input_config}"
config_obj = s3.get_object(Bucket=bucket_name, Key=config_path)
config = json.loads(config_obj['Body'].read())
print(f"Config carregado: {config_path}")

In [ ]:
# Carregar resultado da validação
validation_path = f"{user}/{prefixo_simulado}/artifacts/{input_validation}"
validation_obj = s3.get_object(Bucket=bucket_name, Key=validation_path)
validation_result = json.loads(validation_obj['Body'].read())
print(f"Validação carregada: {validation_path}")
print(f"Arquivos válidos: {len(validation_result.get('arquivos_validos', []))}")

In [ ]:
# Função para calcular métricas de um grupo
def calcular_metricas_grupo(df_grupo):
    """Calcula métricas agregadas para um grupo"""
    metricas = {
        'taxa_conversao': (df_grupo['uso'] > 0).mean() * 100,
        'uso_medio': df_grupo['uso'].mean(),
        'valor_medio': df_grupo['valor'].mean(),
        'usuarios_unicos': df_grupo['id_customer'].nunique()
    }
    return metricas

def padronizar_grupo_controle(df, grupo_controle_config):
    """Padroniza o nome do grupo de controle no DataFrame para o valor do config."""
    variacoes_possiveis = {
        grupo_controle_config.lower().replace('_', '').replace(' ', ''),
        "gc", "grupocontrole", "grupo_controle", "grupo controle", "grupo de controle"
    }
    df['grupo'] = df['grupo'].apply(
        lambda x: grupo_controle_config if str(x).strip().lower().replace('_', '').replace(' ', '') in variacoes_possiveis else x
    )
    return df

In [ ]:
# Processar arquivos e calcular métricas
todas_metricas = []

for arquivo_info in validation_result.get('arquivos_validos', []):
    arquivo_key = arquivo_info['key']
    nome_arquivo = arquivo_info['nome']
    
    print(f"Processando: {nome_arquivo}")
    
    # Ler arquivo do S3
    df = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=arquivo_key)['Body'])
    
    # Padronizar grupo controle se necessário
    grupo_controle = config.get('grupo_controle', 'GC')
    df = padronizar_grupo_controle(df, grupo_controle)
    
    # Processar cada experimento
    for exp_id in df['id_experimento'].unique():
        df_exp = df[df['id_experimento'] == exp_id]
        
        # Processar cada grupo do experimento
        for grupo in df_exp['grupo'].unique():
            df_grupo = df_exp[df_exp['grupo'] == grupo]
            
            # Calcular métricas do grupo
            metricas_grupo = calcular_metricas_grupo(df_grupo)
            
            # Para cada cliente do grupo, criar linhas com as métricas
            for _, cliente_row in df_grupo.iterrows():
                id_customer = cliente_row['id_customer']
                timestamp_original = cliente_row['timestamp']
                
                # Criar uma linha para cada métrica para este cliente
                for nome_metrica, valor_metrica in metricas_grupo.items():
                    todas_metricas.append({
                        'id_experimento': exp_id,
                        'id_customer': id_customer,
                        'grupo': grupo,
                        'metrica': nome_metrica,
                        'valor': valor_metrica,
                        'timestamp': timestamp_original
                    })
    
    print(f"Concluído: {nome_arquivo}")

print(f"Total de métricas calculadas: {len(todas_metricas)}")

In [ ]:
# Salvar métricas em DataFrame e arquivo local
df_metricas = pd.DataFrame(todas_metricas)
df_metricas.to_csv(output_metricas, index=False)
print(f"Métricas salvas localmente: {output_metricas}")
print(f"Linhas: {len(df_metricas)}")
print(f"Colunas: {list(df_metricas.columns)}")

In [ ]:
# Upload das métricas para S3
metricas_s3_path = f"{user}/{prefixo_simulado}/artifacts/{output_metricas}"
s3.upload_file(output_metricas, bucket_name, metricas_s3_path)
print(f"Métricas enviadas para S3: s3://{bucket_name}/{metricas_s3_path}")